In [1]:
import flopy as fp
import numpy as np
import geopandas as gp
import pandas as pd
import os
from osgeo import ogr
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
import matplotlib.pyplot as plt
from flopy.utils.gridgen import Gridgen 
from flopy.utils.gridintersect import GridIntersect
from flopy.utils import Raster
import shapely
from shapely.geometry import Polygon, Point, LineString, MultiLineString, MultiPoint, MultiPolygon,shape
from shapely.strtree import STRtree  
import glob
import sys

sys.path.append("../../modules")
# hand made functions
from Rouss1 import *
from Rouss2 import *

sys.path.append("../../Model_val/upscalling/")
from upscalling_functions import *

flopy is installed in C:\Users\emmal\Anaconda3\lib\site-packages\flopy


In [2]:
#geo parameters
MNT_path = "../../data/surfaces_geoter/TOPO (inf_0 en mer).ASC"
x0,y0,x1,y1 = get_MNTbbox(MNT_path)

#Z
nlay = 5

#X
x_seashore = 705e3
Lx_1 = x_seashore-x0
Lx_2 = x1 - x_seashore
res_1 = 300
res_2 = 600
ncol1 = int(Lx_1/res_1)
ncol2 = int(Lx_2/res_2)
ncol = ncol1+ncol2

delr = np.ones(ncol)*res_1
delr[ncol1:ncol1+1] *=1.5
delr[ncol1+1:] *= 2


#Y
Ly = y1-y0
res = 300
nrow = int(Ly/res)
delc = np.ones(nrow)*res

grid = fp.discretization.StructuredGrid(delc,delr,xoff=x0,yoff=y0) # create a grid identical to the dis package, will be used
                                                                   # to pre-process data
idomain=np.zeros([nrow*ncol])

In [3]:
# load top data
MNT = Raster.load(MNT_path)

grid = fp.discretization.StructuredGrid(delc,delr,xoff=x0,yoff=y0) # create a grid identical to the dis package, will be used
                                                                 # to pre-process data

top = MNT.resample_to_grid(grid.xcellcenters,
                                grid.ycellcenters,
                                band = MNT.bands[0],
                                method="nearest")

In [4]:
# load botom based on the differents surfaces
folder_path = "../../data/surfaces_geoter/"
surfaces = []
for file in glob.glob(os.path.join(folder_path, '*.asc')):
    Rast = Raster.load(file)
    surfaces.append(Rast.resample_to_grid(grid.xcellcenters,
                                grid.ycellcenters,
                                band = Rast.bands[0],
                                method="nearest"))
Q = surfaces[2]
PC = surfaces[0]
PMS = surfaces[1]
BOT = PMS.copy()
BOT[PMS==-9999] = PC[PMS==-9999]
BOT[PC==-9999] = Q[PC==-9999]
BOT[BOT==-9999] = top[BOT==-9999] - 50

BOT[(top-BOT)<=10] = top[(top-BOT)<=10] - 10 # minimum of 10 m thickness to avoid bug and numerical issues

In [5]:
# Define the domain
R_path = "../../data/shp/limiteModeleRoussillon_ext_poly.shp"
R = gp.read_file(R_path) # import shapefile with geopandas

# cells inside the aquifer become active and return the lst of the cellids
lst_domain = gp2idomain(R,grid,idomain,area=0) # all the cells of the model

In [6]:
idomain = idomain.reshape(nrow,ncol)

In [7]:
write_pickle("working/export_dis",[top,BOT,idomain,delr,delc])